# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [2]:
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [4]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [5]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/ces-grands-parents-chicouf-tres-mobilises-et-sollicites-pour-garder-les-enfants-11624846',
  'Ces grands-parents "chicouf", très mobilisés et sollicités pour garder les enfants'],
 ['https://www.rtbf.be/article/coupe-de-belgique-un-choc-genk-anderlecht-en-8e-tirage-clement-pour-le-standard-et-l-union-11624891',
  "Coupe de Belgique : un choc Genk - Anderlecht en 8e, tirage clément pour le Standard et l'Union"],
 ['https://www.rtbf.be/article/affaire-epstein-le-roi-charles-iii-a-lance-un-processus-formel-pour-retirer-ses-titres-au-prince-andrew-11624863',
  'Affaire Epstein\xa0: le roi Charles III a lancé un "processus formel" pour retirer ses titres au prince Andrew'],
 ['https://www.rtbf.be/article/masters-1000-de-paris-premier-quart-pour-sinner-a-trois-succes-du-trone-de-n-1-mondial-11624854',
  'Masters 1000 de Paris\xa0: premier quart pour Sinner, à trois succès du trône de N.1 mondial'],
 ['https://www.rtbf.be/article/l-indexation-automatique-des-sal

## Création d'un dataframe avec les liens et les titres des articles


In [6]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/ces-grands-parents...,"Ces grands-parents ""chicouf"", très mobilisés e..."
1,https://www.rtbf.be/article/coupe-de-belgique-...,Coupe de Belgique : un choc Genk - Anderlecht ...
2,https://www.rtbf.be/article/affaire-epstein-le...,Affaire Epstein : le roi Charles III a lancé u...
3,https://www.rtbf.be/article/masters-1000-de-pa...,Masters 1000 de Paris : premier quart pour Sin...
4,https://www.rtbf.be/article/l-indexation-autom...,L’indexation automatique des salaires pourrait...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Comme à la maison
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Météo 20h00
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le 19h30
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Bones S12


In [7]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [8]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

C:\Users\fabio\AppData\Local\Temp\ipykernel_6636\3125040403.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  article_url = df.iloc[0][0]


Belgique
Pendant les vacances scolaires, certains grands-parents se mobilisent pour garder leurs petits enfants. Un moment familial précieux qui prend cependant aussi beaucoup d'énergie. Beaucoup avouent dès lors un certain soulagement lorsqu'ils voient les enfants repartir.   "Quand on les voit, ils nous sautent dans les bras. C'est super agréable. Mais quand ils repartent c'est une tornade qui s'en va" , confie Raffaela.
Pour les parents, pouvoir être aidés par leurs propres parents afin de s'occuper des enfants n'est pas donné à tout le monde. C'est ce qu'explique Madeleine Guyot, directrice générale de la Ligue des familles :  "Dans notre baromètre 2024, 42% des parents qui nous indiquent qu'ils n'ont pas de solution auprès de proches pour les aider dans le cadre de l'organisation familiale. Seulement un faible pourcentage, 15% des parents nous ont indiqués qu'ils peuvent compter sur l'aide de leurs proches, sans toutefois préciser s'il s'agit de leurs parents."
Mots clés de l'arti

### Nettoyage du texte à l'aide d'expressions régulières

In [9]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Pendant les vacances scolaires, certains grands-parents se mobilisent pour garder leurs petits enfants. Un moment familial précieux qui prend cependant aussi beaucoup d'énergie. Beaucoup avouent dès lors un certain soulagement lorsqu'ils voient les enfants repartir.   "Quand on les voit, ils nous sautent dans les bras. C'est super agréable. Mais quand ils repartent c'est une tornade qui s'en va" , confie Raffaela.
Pour les parents, pouvoir être aidés par leurs propres parents afin de s'occuper des enfants n'est pas donné à tout le monde. C'est ce qu'explique Madeleine Guyot, directrice générale de la Ligue des familles :  "Dans notre baromètre 2024, 42% des parents qui nous indiquent qu'ils n'ont pas de solution auprès de proches pour les aider dans le cadre de l'organisation familiale. Seulement un faible pourcentage, 15% des parents nous ont indiqués qu'ils peuvent compter sur l'aide de leurs proches, sans toutefois préciser s'il s'agit de leurs parents."
Mots clés de l'article
Copyr

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:4: SyntaxWarning: invalid escape sequence '\s'
C:\Users\fabio\AppData\Local\Temp\ipykernel_6636\3799268790.py:2: SyntaxWarning: invalid escape sequence '\('
  clean_content = re.sub("\([^\)]+\)", " ", content)
C:\Users\fabio\AppData\Local\Temp\ipykernel_6636\3799268790.py:4: SyntaxWarning: invalid escape sequence '\s'
  clean_content = re.sub("\s+", " ", clean_content)


### Création d'un fichier avec le contenu de l'article


In [10]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp